In [ ]:
pip install neo4j

In [ ]:
pip install pandas

In [41]:
import pandas as pd
import os

In [42]:
from neo4j import GraphDatabase

# Replace these with your actual AuraDB connection details
uri = "neo4j+s://b480f347.databases.neo4j.io"
username = "neo4j"
password = "8erltLOKbpg68_LbDjbZtsHQC-nAKV-advvPvHLdpII"

driver = GraphDatabase.driver(uri, auth=(username,password))


In [43]:
query = "MATCH (n) RETURN COUNT(n)"  # Query to check connection

In [ ]:
with driver.session() as session:
  result = session.run(query)
  print(result.single()["COUNT(n)"])

In [ ]:
from google.colab import drive  # Mounting drive to Google Colab. Ignore if not using Colab
drive.mount('/content/drive')

In [46]:
# Specify the directory path for the CSV files
csv_dir = '/content/drive/MyDrive/Cloud Computing EC528/Parsed SBOM/v1.18.20/'  # Replace with your directory path

In [47]:
# Load the CSV files into pandas DataFrames
packages_data = pd.read_csv(csv_dir + 'v1.18.20_packages_output.csv')
spdx_data = pd.read_csv(csv_dir + 'v1.18.20_spdx_output.csv')
relationships_data = pd.read_csv(csv_dir+'v1.18.20_relationships_output.csv')

def create_nodes(tx, data, label, identifier_key):
    for index, row in data.iterrows():
        node_data = dict(row)
        tx.run(
            f"CREATE (node:{label} {{{identifier_key}: $value}})",
            value=node_data[identifier_key]
        )

def create_nodes_packages(tx, data, label, identifier_key):
    for index, row in data.iterrows():
        node_data = dict(row)
        # Create the node with properties from the CSV columns
        query = (
            f"CREATE (node:{label} {{"
            f"{identifier_key}: $value, "
            f"SPDXID: $SPDXID, "
            f"PackageName: $PackageName, "
            f"PackageDownloadLocation: $PackageDownloadLocation, "
            f"FilesAnalyzed: $FilesAnalyzed, "
            f"PackageLicenseConcluded: $PackageLicenseConcluded, "
            f"PackageLicenseDeclared: $PackageLicenseDeclared, "
            f"PackageCopyrightText: $PackageCopyrightText, "
            f"PackageChecksum: $PackageChecksum"
            # Add more properties as needed, and remove the trailing comma
            f"}})"
        )
        tx.run(query, {
            'value': node_data[identifier_key],
            'SPDXID': node_data['SPDXID'],
            'PackageName': node_data['PackageName'],
            'PackageDownloadLocation': node_data['PackageDownloadLocation'],
            'FilesAnalyzed': node_data['FilesAnalyzed'],
            'PackageLicenseConcluded': node_data['PackageLicenseConcluded'],
            'PackageLicenseDeclared': node_data['PackageLicenseDeclared'],
            'PackageCopyrightText': node_data['PackageCopyrightText'],
            'PackageChecksum': node_data['PackageChecksum']
            # Add more property values as needed
        })

def create_nodes_spdx(tx, data, label, identifier_key):
    for index, row in data.iterrows():
        node_data = dict(row)
        # Create the node with properties from the CSV columns
        query = (
            f"CREATE (node:{label} {{"
            f"{identifier_key}: $value, "
            f"SPDXID: $SPDXID, "
            f"FileName: $FileName, "
            f"FileChecksum: $FileChecksum, "
            f"LicenseConcluded: $LicenseConcluded, "
            f"LicenseInfoInFile: $LicenseInfoInFile, "
            f"FileCopyrightText: $FileCopyrightText"
            # Add more properties as needed, and remove the trailing comma
            f"}})"
        )
        tx.run(query, {
            'value': node_data[identifier_key],
            'SPDXID': node_data['SPDXID'],
            'FileName': node_data['FileName'],
            'FileChecksum': node_data['FileChecksum'],
            'LicenseConcluded': node_data['LicenseConcluded'],
            'LicenseInfoInFile': node_data['LicenseInfoInFile'],
            'FileCopyrightText': node_data['FileCopyrightText']
            # Add more property values as needed
        })



# Create nodes for SPDX data using SPDXID as the label
with driver.session() as session:
    session.write_transaction(create_nodes_spdx, spdx_data, 'SPDXNode', 'SPDXID')

# Create nodes for packages data using SPDXID as the label
with driver.session() as session:
    session.write_transaction(create_nodes_packages, packages_data, 'PackageNode', 'SPDXID')

# Create Relationships
with driver.session() as session:
    for index, row in relationships_data.iterrows():
        relationship_data = dict(row)
        session.run(
            "MATCH (source {SPDXID: $source}), (target {SPDXID: $target}) "
            "CREATE (source)-[:RELATIONSHIP_TYPE {type: $types}]->(target)",
            source=relationship_data['RelationshipType'],  #the Source/RelationshipType column in the .csv file were mislabeled and interchanged
            target=relationship_data['Target'],
            types=relationship_data['Source']
        )


<ipython-input-47-97aa7e077513>:76: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_nodes_spdx, spdx_data, 'SPDXNode', 'SPDXID')
<ipython-input-47-97aa7e077513>:80: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_nodes_packages, packages_data, 'PackageNode', 'SPDXID')


In [48]:
# Define the folder path where your CSV files are located
csv_folder_path = '/content/drive/MyDrive/Cloud Computing EC528/Parsed SBOM/v1.18.20'

# Extract the folder name from the path
folder_name = os.path.basename(csv_folder_path)

In [49]:
# Define a function to create a folder node and connect nodes to it
def create_folder_and_connect_nodes(tx, folder_name):
    # Create the folder node
    tx.run("CREATE (folder:Folder {name: $folder_name})", folder_name=folder_name)

    # Connect all nodes to the folder node
    tx.run(
        "MATCH (node) WHERE NOT node:Folder "
        "WITH node "
        "MATCH (folder:Folder {name: $folder_name}) "
        "CREATE (node)-[:BELONGS_TO]->(folder)",
        folder_name=folder_name
    )

# Execute the folder creation and connection
with driver.session() as session:
    session.write_transaction(create_folder_and_connect_nodes, folder_name)

<ipython-input-49-52a84ddf99b7>:17: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_folder_and_connect_nodes, folder_name)
